## SP500 data availability analysis (exclude tickers to maximize usable dates)

This notebook analyzes **data coverage** for the `SP500` universe in `tickers.json`.

Goal: find a small set of tickers to exclude so that the remaining panel has the **maximum number of dates** where **all remaining tickers AND the selected market** have prices (strict intersection).

Notes:
- Fetch step stores **raw adjusted close prices** in the local `cache/`.
- Compute step uses only cached prices unless you set `ALLOW_FETCH=True`.
- The greedy exclusion algorithm is fast and explainable; it may not be globally optimal but works well in practice.



In [1]:
from pathlib import Path
import json

import pandas as pd

import data_preloading_tools as DPT

# --- config ---
UNIVERSE = "SP500"
MARKET = "SPY"
ALLOW_FETCH = True
BATCH_SIZE = 50
DURATION = "15 Y"
BAR_SIZE = "1 day"

PANEL_CACHE_PATH = Path("cache") / "companies" / f"{UNIVERSE}_prices_panel.pkl"
MARKET_CACHE_PATH = Path("cache") / "mkt" / f"{MARKET}_prices.pkl"

# --- load universe ---
spx = DPT.load_universe_tickers("tickers.json", UNIVERSE)
print("universe", UNIVERSE, "tickers", len(spx))

# --- load caches first ---
comp_px = DPT.load_prices_panel(PANEL_CACHE_PATH)
mkt_px = DPT.load_market_prices(MARKET_CACHE_PATH)

# --- optionally fetch/build caches ---
if ALLOW_FETCH:
    from ib_insync import util
    import ib_connection as IBC

    util.startLoop()
    ibc, ib = IBC.quick_connect()

    if comp_px is None:
        fr = DPT.build_companies_prices_panel_from_ib(
            ib,
            tickers=spx,
            duration=DURATION,
            bar_size=BAR_SIZE,
            batch_size=BATCH_SIZE,
            progress=True,
        )
        comp_px = fr.prices
        DPT.save_pickle(PANEL_CACHE_PATH, comp_px)
        if fr.invalid_tickers:
            print("invalid tickers (excluded):", len(fr.invalid_tickers))
            print(fr.invalid_tickers[:50])

    if mkt_px is None:
        mkt_px = DPT.fetch_market_prices_from_ib(
            ib,
            market=MARKET,
            duration=DURATION,
            bar_size=BAR_SIZE,
            progress=True,
        )
        DPT.save_pickle(MARKET_CACHE_PATH, mkt_px)

# --- run analysis (cache-only unless ALLOW_FETCH=True) ---
if (comp_px is None) or (mkt_px is None):
    print("Missing caches. Set ALLOW_FETCH=True to build them.")
else:
    present = [t for t in spx if t in comp_px.columns]
    missing = [t for t in spx if t not in comp_px.columns]
    print("present tickers in panel:", len(present))
    print("missing tickers (not in panel):", len(missing))

    cov = DPT.coverage_table(comp_px, spx)
    print("worst coverage tickers (bottom 50):")
    display(cov.head(50))

    result = DPT.greedy_exclude_for_max_dates(comp_px, mkt_px, tickers=present)
    print("kept:", len(result["kept"]))
    print("removed:", len(result["removed"]))
    print("strict valid dates:", len(result["valid_dates"]))
    if len(result["valid_dates"]) > 0:
        print("valid date range:", result["valid_dates"].min(), "->", result["valid_dates"].max())

    print("first 30 removed tickers:")
    print(result["removed"][:30])

    print("greedy history (last 20 steps):")
    display(result["history"].tail(50))



universe SP500 tickers 503
present tickers in panel: 501
missing tickers (not in panel): 2
worst coverage tickers (bottom 50):


,non_nan,start,end
TPL,0,NaT,NaT
LIN,0,NaT,NaT
MNST,0,NaT,NaT
Q,47,2025-10-27,2026-01-02
PSKY,103,2025-08-07,2026-01-02
SNDK,223,2025-02-13,2026-01-02
SW,375,2024-07-08,2026-01-02
ODFL,416,2024-05-07,2026-01-02
GEV,444,2024-03-27,2026-01-02
SOLV,445,2024-03-26,2026-01-02


kept: 501
removed: 0
strict valid dates: 0
first 30 removed tickers:
[]
greedy history (last 20 steps):


,n_tickers,n_valid_dates,removed_ticker,gain_dates
0,501,0,None,0


In [ ]:
# Filter rule: keep tickers with >= MIN_VALID_DATES non-NaN price points
MIN_VALID_DATES = 2000
DO_FILTER_WRITEBACK = True

if DO_FILTER_WRITEBACK and (comp_px is not None):
    # NOTE: reload is needed if you edited data_preloading_tools.py during this notebook session
    import importlib
    importlib.reload(DPT)

    filt = DPT.filter_and_cache_sp500_by_min_valid_dates(
        comp_px,
        sp500_tickers=spx,
        min_valid_dates=MIN_VALID_DATES,
        out_universe_key="SP500_filtered",
    )
    print("SP500_filtered kept:", len(filt["kept"]))
    print("min non_nan among kept:", filt.get("min_non_nan_kept"))
    print("filtered cache:", filt["out_panel_path"], "shape", filt["filtered_panel_shape"])

    print("bottom 20 by non_nan among SP500_filtered KEPT:")
    display(filt["stats_kept"].head(20))

    print("bottom 20 by non_nan among REMOVED (< threshold):")
    display(filt["stats_removed"].head(20))



SP500_filtered kept: 463
filtered cache: cache\companies\SP500_filtered_prices_panel.pkl shape (3769, 463)
bottom 20 by non_nan among SP500_filtered candidates:


,non_nan,start,end
TPL,0,NaT,NaT
LIN,0,NaT,NaT
MNST,0,NaT,NaT
Q,47,2025-10-27,2026-01-02
PSKY,103,2025-08-07,2026-01-02
SNDK,223,2025-02-13,2026-01-02
SW,375,2024-07-08,2026-01-02
ODFL,416,2024-05-07,2026-01-02
GEV,444,2024-03-27,2026-01-02
SOLV,445,2024-03-26,2026-01-02


In [3]:
# Optional: write back a filtered SP500 universe into tickers.json
# Set to True only if you want to persist the recommendation.
WRITE_BACK = False
NEW_KEY = "SP500_filtered"

if WRITE_BACK and (comp_px is not None) and (mkt_px is not None):
    kept = result["kept"]
    cfg2 = json.loads(Path("tickers.json").read_text(encoding="utf-8"))
    cfg2[NEW_KEY] = {
        "description": f"SP500 filtered to maximize strict intersection vs {MARKET}",
        "tickers": kept,
    }
    Path("tickers.json").write_text(json.dumps(cfg2, indent=4), encoding="utf-8")
    print("wrote", NEW_KEY, "tickers", len(kept))

